# Challenge 05/12/2024

Challenge instructions [here](https://adventofcode.com/2024/day/5)

## Highlights & Notes

- Goal: identify page ordering rules that are correct (according to the rules)
  - Then find the middle page number for all of these and add them together


## Setup & Imports


In [2]:
import numpy as np
import sys
import os

# Get utility functions
sys.path.append(os.path.abspath('../utils'))
from utils import read_input_file, remove_newline_char, split_lines

# Quick ANSI color code shortcuts
r = "\033[31m";y = "\033[33m";g = "\033[32m";b = "\033[34m";e = "\033[0m"

## Part 1


In [11]:
# Open file, read until an empty line and store each line as a tuple (format "X|Y"),
# then read after this empty line and store each line as a list (format "X,Y,Z,...")
def get_input(filename: str = "demo.txt"):
    with open(filename, 'r') as f:
        lines = f.readlines()
    # Find the index of the empty line
    empty_line_index = lines.index('\n')
    # Split the lines into two lists
    ordering_rules = [line.strip() for line in lines[:empty_line_index]]
    update_lines = [line.strip() for line in lines[empty_line_index + 1:]]
    return ordering_rules, update_lines

def parse_ordering_rules(ordering_rules):
    # Each ordering rule is in the format "X|Y"
    ordering_tuples = []
    for rule in ordering_rules:
        x, y = rule.split('|')
        ordering_tuples.append((int(x), int(y)))
    return ordering_tuples

def parse_updates(update_lines):
    # Each update line is in the format "X,Y,Z,..."
    updates = []
    for line in update_lines:
        pages = [int(page) for page in line.split(',') if page]
        updates.append(pages)
    return updates

def is_update_valid(update, ordering_tuples):
    pages_in_update = set(update)
    # Map each page to its index in the update
    page_positions = {page: idx for idx, page in enumerate(update)}
    # Check the ordering rules relevant to the pages in the update
    for x, y in ordering_tuples:
        if x in pages_in_update and y in pages_in_update:
            if page_positions[x] > page_positions[y]:
                # Rule violated: x appears after y
                return False
    return True

def get_middle_page(update):
    n = len(update)
    mid_index = n // 2
    return update[mid_index]

# Main execution
ordering_rules, update_lines = get_input("input.txt")
ordering_tuples = parse_ordering_rules(ordering_rules)
updates = parse_updates(update_lines)

valid_updates = []
invalid_updates = []
total = 0
for update in updates:
    if is_update_valid(update, ordering_tuples):
        valid_updates.append(update)
        middle_page = get_middle_page(update)
        total += middle_page
    else:
        invalid_updates.append(update)

print(f"Total sum of middle page numbers: {total}")
print(f"Number of invalid updates: {len(invalid_updates)}")

Total sum of middle page numbers: 5275
Number of invalid updates: 107


## Part 2

Instructions [here](https://adventofcode.com/2024/day/2#part2)


In [12]:
from collections import defaultdict, deque

def topological_sort(pages, ordering_tuples):
    # Build the graph
    graph = defaultdict(list)
    indegree = defaultdict(int)
    
    pages_set = set(pages)
    for page in pages:
        indegree[page] = 0  # Initialize indegree to zero

    for x, y in ordering_tuples:
        if x in pages_set and y in pages_set:
            graph[x].append(y)
            indegree[y] += 1
    
    # Queue for nodes with zero indegree
    queue = deque([page for page in pages if indegree[page] == 0])
    sorted_pages = []

    while queue:
        u = queue.popleft()
        sorted_pages.append(u)
        for v in graph[u]:
            indegree[v] -= 1
            if indegree[v] == 0:
                queue.append(v)
    
    if len(sorted_pages) != len(pages):
        # Cycle detected, cannot sort
        return None
    return sorted_pages

invalid_total = 0
for update in invalid_updates:
    ordered_update = topological_sort(update, ordering_tuples)
    if ordered_update:
        middle_page = get_middle_page(ordered_update)
        invalid_total += middle_page
    else:
        # Cannot reorder due to cycles, skip this update
        pass

print(f"Total sum of middle page numbers after reordering: {invalid_total}")

Total sum of middle page numbers after reordering: 6191


## O1-preview explanation of `topological-sort()`


### Overview

The `topological_sort()` function is used to reorder the pages in each update so that they respect the ordering rules provided. If the pages can be rearranged to satisfy all the rules without conflicts, the function returns the reordered list. If not, it returns `None`.

This rearrangement is necessary because some updates originally violate the ordering rules. By reordering them, we can include them in our calculations.

### What is Topological Sorting?

Topological sorting is a method used in computer science to linearly order the nodes of a **directed acyclic graph (DAG)** such that for every directed edge from node A to node B, node A comes before node B in the ordering.

In simpler terms, if you have a set of tasks with dependencies (some tasks must be done before others), topological sorting gives you an order to perform the tasks without violating any dependencies.

### Understanding the `topological_sort()` Function

Here's a step-by-step explanation of what the `topological_sort()` function does:

1. **Build the Graph**:

   - **Nodes**: Each page number in the update is a node in the graph.
   - **Edges**: An edge from page X to page Y means page X must come before page Y (based on the ordering rules).
   - **Graph Representation**: We use a dictionary (`graph`) to represent the adjacency list of the graph.

2. **Initialize Indegrees**:

   - **Indegree**: The number of edges coming into a node.
   - **Purpose**: Indegrees help us identify nodes with no dependencies (`indegree` of zero), which can be placed at the beginning of the ordering.
   - We initialize a dictionary (`indegree`) with all pages having an indegree of zero.

3. **Populate the Graph and Indegrees**:

   - We iterate over all ordering rules that involve pages in the current update.
   - **For each rule (X must come before Y)**:
     - Add Y to the list of adjacent nodes for X in the graph (`graph[X].append(Y)`).
     - Increment the indegree of Y (`indegree[Y] += 1`), because there's one more page that must come before Y.

4. **Initialize the Processing Queue**:

   - We create a queue (`queue`) of all nodes with an indegree of zero. These are the pages with no prerequisites and can be placed first in the ordering.

5. **Process the Nodes**:

   - We initialize an empty list (`sorted_pages`) to store the pages in the correct order.
   - **While the queue is not empty**:
     - Remove a page (`u`) from the front of the queue.
     - Add `u` to `sorted_pages`.
     - For each page (`v`) that comes after `u` (i.e., `for v in graph[u]`):
       - Decrement the indegree of `v` (`indegree[v] -= 1`).
       - If `indegree[v]` becomes zero, add `v` to the queue.

6. **Check for Cycles**:

   - After processing all nodes:
     - If the length of `sorted_pages` is less than the number of pages in the update, it means there is a cycle (conflict) in the ordering rules, and it's impossible to sort the pages without violating the rules.
     - In this case, the function returns `None`.

7. **Return the Sorted Pages**:

   - If no cycles are detected, the function returns `sorted_pages`, which is a valid ordering of the pages according to the ordering rules.

### Simplified Explanation with an Example

Let's consider a simple update with pages `[1, 2, 3]` and ordering rules:

- `1|2`: Page 1 must come before page 2.
- `2|3`: Page 2 must come before page 3.

**Step-by-Step Execution**:

1. **Build the Graph and Indegrees**:

   - **Graph**:
     - `graph[1] = [2]`
     - `graph[2] = [3]`
   - **Indegrees**:
     - `indegree[1] = 0` (no pages must come before page 1)
     - `indegree[2] = 1` (page 1 must come before page 2)
     - `indegree[3] = 1` (page 2 must come before page 3)

2. **Initialize the Queue**:

   - Pages with `indegree` zero: `[1]`
   - `queue = deque([1])`

3. **Process the Nodes**:

   - **Process Page 1**:
     - Remove `1` from the queue.
     - Add `1` to `sorted_pages`: `[1]`
     - For `v` in `graph[1]` (which is `[2]`):
       - Decrement `indegree[2]` to `0`.
       - Add `2` to the queue.
   - **Process Page 2**:
     - Remove `2` from the queue.
     - Add `2` to `sorted_pages`: `[1, 2]`
     - For `v` in `graph[2]` (which is `[3]`):
       - Decrement `indegree[3]` to `0`.
       - Add `3` to the queue.
   - **Process Page 3**:
     - Remove `3` from the queue.
     - Add `3` to `sorted_pages`: `[1, 2, 3]`

4. **Check for Cycles**:

   - The length of `sorted_pages` is 3, which matches the number of pages.
   - No cycles detected.

5. **Return the Sorted Pages**:

   - The function returns `[1, 2, 3]`, a valid ordering.

### Handling Conflicts (Cycles)

If there's a circular dependency, like `1|2` and `2|1`, it creates a cycle.

**Example with Conflict**:

- Pages: `[1, 2]`
- Ordering Rules:
  - `1|2`: Page 1 must come before page 2.
  - `2|1`: Page 2 must come before page 1.

**Processing**:

- Indegrees:
  - `indegree[1] = 1` (due to `2|1`)
  - `indegree[2] = 1` (due to `1|2`)
- Since there are no pages with `indegree` zero, the queue is empty from the start.
- The `sorted_pages` list remains empty, indicating a cycle.
- The function returns `None`.

### Applying to Incorrect Updates

In your problem, for updates that were originally incorrect:

- **Attempt to Reorder**:

  - Use `topological_sort()` to try to find a valid order that satisfies all the ordering rules for the pages in the update.

- **Sum Middle Pages of Reordered Updates**:
  - If reordering is successful, calculate the middle page of the reordered update and add it to your total.
  - If reordering isn't possible (due to cycles), the update is skipped, as it can't be corrected without violating the rules.

### Why Use Topological Sorting?

Topological sorting allows us to:

- **Find a Valid Order**:

  - It rearranges the pages to respect all dependencies (ordering rules).

- **Detect Impossible Cases**:
  - If dependencies conflict (cycles), it identifies that reordering is impossible.

### Conclusion

The `topological_sort()` function is essential for reordering the incorrect updates so they become valid according to the rules. By understanding that it constructs a dependency graph and orders the pages while respecting these dependencies, you can see how it reorganizes the pages to meet the constraints.

**Key Takeaways**:

- **Nodes**: Pages in the update.
- **Edges**: Ordering rules (X must come before Y).
- **Indegree**: The number of dependencies a page has.
- **Queue**: Starts with pages that have no dependencies.
- **Processing**: Remove pages with no dependencies and reduce the indegrees of dependent pages.
- **Cycle Detection**: If we can't process all pages, there's a cycle (conflict).

I hope this clarifies how the `topological_sort()` function works! Let me know if you have any more questions.
